In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [3]:
import pandas as pd
import numpy as np
import altair as alt
import altair_saver
import glob
import os
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

In [4]:
def personal():
    return {
        'config': {
            'view': {
                'height': 400,
                'width': 600,
            },
            'range': {
                'category': {'scheme': 'set1'},
                'ordinal': {'scheme': 'set1'},
            },
            'legend': {
                'labelLimit': 0,
            },
            'background': 'white',
#             'mark': {
#                 'clip': True,
#             },
        }
    }
alt.themes.register('personal', personal)
alt.themes.enable('personal')

ThemeRegistry.enable('personal')

In [5]:
def load_jobs(pattern, subdir='exploration'):    
    jobs = glob.glob(f'results/{subdir}/{pattern}')
    results = []
    for job in jobs:
        name = os.path.basename(os.path.normpath(job))
        train_data = pd.read_csv(job + '/train.csv')
        train_data['eval'] = False
        test_data = pd.read_csv(job + '/test.csv')
        test_data['eval'] = True
        data = pd.concat([train_data, test_data], sort=False)
        data['name'] = name
        results.append(data)
    df = pd.concat(results, sort=False)
    return df.reset_index(drop=True)

In [18]:
data = pd.concat([load_jobs('point-mass_regular'),
                  load_jobs('point-mass_clipvalue'),
                  load_jobs('point-mass_clipvalue_exptemp1'),
                  load_jobs('point-mass_noexplore*'),
                 ], sort=False)

subset = data
subset = subset[subset['eval'] == True]
subset = subset[subset['episode'] <= 150]
alt.Chart(subset).mark_line().encode(
    x='episode',
    y='rolling_mean_score:Q',
    color='name',
    detail='eval'
).transform_window(
    rolling_mean_score='mean(score)',
    frame=[-20, 20]
).transform_window(
    rolling_mean_novelty='mean(novelty_score)',
    frame=[-20, 20]
)

alt.Chart(...)

In [13]:
data

,episode,novelty_score,score,eval,name
0,1.0,578.977453,0.000000e+00,False,point-mass_ptemt1_testtemp1
1,2.0,561.277605,1.689060e-34,False,point-mass_ptemt1_testtemp1
2,3.0,466.144192,1.428059e+00,False,point-mass_ptemt1_testtemp1
3,4.0,386.458881,3.350720e-231,False,point-mass_ptemt1_testtemp1
4,5.0,232.650884,0.000000e+00,False,point-mass_ptemt1_testtemp1
...,...,...,...,...,...
1265,631.0,33.183463,6.408994e+02,True,point-mass_noexplore_temp1
1266,632.0,50.727364,4.011296e+02,True,point-mass_noexplore_temp1
1267,633.0,76.923474,4.112023e+02,True,point-mass_noexplore_temp1
1268,634.0,59.564465,2.029948e+01,True,point-mass_noexplore_temp1
